# Information Withholding by Pleiss et al. - Adult data

This notebook contains the implementation of the post-processing algorithm introduced in [On fairness and calibration](https://dl.acm.org/doi/10.5555/3295222.3295319) by Pleiss et al. (2017) as part of the IBM AIF360 fairness tool box github.com/IBM/AIF360.

The migitation method achieves a relaxed version of Equalised Odds while maintaining Calibration by withholding information. In particular a proportion of the advantaged group is predicted according to the base rate without considering the model inputs. This preserves Calibration but allows us to bring the error rates for the two classes closer together.

This method is attractive in that it achieves one notion of fairness and approximately achieves another. However, like the intervention of Hardt et al. it introduces randomness into decision making that might not be compatible with individual notions of fairness. Furthermore the method requires as input calibrated classifiers, it does not offer a way to achieve Calibration, only to preserve it.


In [31]:
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
from aif360.datasets import StandardDataset
from aif360.algorithms.postprocessing.calibrated_eq_odds_postprocessing import (
    CalibratedEqOddsPostprocessing,
)
from fairlearn.metrics import equalized_odds_difference
from helpers.metrics import accuracy, calibration
from helpers.plot import group_bar_plots

In [32]:
from helpers import export_plot

## Load data

We have committed preprocessed data to the repository for reproducibility and we load it here. Check out hte preprocessing notebook for details on how this data was obtained.

In [33]:
artifacts_dir = Path("../../../artifacts")

In [34]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../../artifacts")

In [35]:
data_dir = artifacts_dir / "data" / "adult"

train = pd.read_csv(data_dir / "processed" / "train-one-hot.csv")
val = pd.read_csv(data_dir / "processed" / "val-one-hot.csv")
test = pd.read_csv(data_dir / "processed" / "test-one-hot.csv")

In order to process data for our fairness intervention we need to define special dataset objects which are part of every intervention pipeline within the IBM AIF360 toolbox. These objects contain the original data as well as some useful further information, e.g., which feature is the protected attribute as well as which column corresponds to the label.

In [36]:
train_sds = StandardDataset(
    train,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
test_sds = StandardDataset(
    test,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
val_sds = StandardDataset(
    val,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
index = train_sds.feature_names.index("sex")

Define which binary value goes with the (un-)privileged group

In [37]:
privileged_groups = [{"sex": 1.0}]
unprivileged_groups = [{"sex": 0.0}]

## Load original model

For maximum reproducibility we can also load the baseline model from disk, but the code used to train can be found in the baseline model notebook.

In [38]:
baseline_model = joblib.load(
    artifacts_dir / "models" / "finance" / "baseline.pkl"
)

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning:

Trying to unpickle estimator LabelBinarizer from version 0.23.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning:

Trying to unpickle estimator MLPClassifier from version 0.23.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.



Get predictions for the validation and test data

In [39]:
bl_test_probs = baseline_model.predict_proba(test.drop("salary", axis=1))[:, 1]
test_sds_pred = test_sds.copy(deepcopy=True)
test_sds_pred.scores = bl_test_probs.reshape(-1, 1)
bl_test_pred = bl_test_probs > 0.5

bl_val_probs = baseline_model.predict_proba(val.drop("salary", axis=1))[:, 1]
val_sds_pred = val_sds.copy(deepcopy=True)
val_sds_pred.scores = bl_val_probs.reshape(-1, 1)
bl_val_pred = bl_val_probs > 0.5

## Equal opportunity

We first address equal opportunity which is achieved by setting the cost_contraint parameter method accordingly when setting up the intervention. We then learn the intervention procedure based on the true and predicted labels of the validation data. Subsequently, we apply the learnt intervention to the predictions of the test data and analyse the outcomes for fairness and accuracy.

In [40]:
cost_constraint = "fnr"

In [41]:
# Learn parameters to equal opportunity and apply to create a new dataset
cpp = CalibratedEqOddsPostprocessing(
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
    cost_constraint=cost_constraint,
    seed=np.random.seed(),
)
cpp = cpp.fit(val_sds, val_sds_pred)

Apply intervention to testing data.

In [42]:
test_sds_pred_tranf = cpp.predict(test_sds_pred)
test_probs = test_sds_pred_tranf.scores.flatten()
test_pred = test_probs > 0.5

Analyse accuracy and fairness

### Calibration

Pleiss et al.'s intervention performs on optimisation over the space of calibration-preserving modifications. Hence, if the original unfair model is calibrated, the corrected model will be as well. This phenomenon is displayed in the figure below.

In [43]:
cali_scatter = calibration_plots(
    test.salary,
    [test_probs, test_sds_pred_tranf.scores.flatten()],
    test.sex,
    10,
)

In [44]:
bl_acc = accuracy(test.salary, bl_test_pred)
acc = accuracy(test.salary, test_pred)

bl_eo = equalized_odds_difference(
    test.salary, bl_test_pred, sensitive_features=test.sex,
)
eo = equalized_odds_difference(
    test.salary, test_pred, sensitive_features=test.sex,
)

bl_calib = calibration(test.salary, bl_test_probs, test.sex)
calib = calibration(test.salary, test_probs, test.sex)

print(f"Baseline accuracy: {bl_acc:.3f}")
print(f"Accuracy: {acc:.3f}")

print(f"Baseline equalised odds difference: {bl_eo:.3f}")
print(f"Equalised odds difference: {eo:.3f}")

print(f"Baseline calibration: {bl_calib:.3f}")
print(f"Calibration: {calib:.3f}")

In [ ]:
eopp_bar = group_bar_plots(
    np.concatenate([bl_test_probs, test_probs]),
    np.tile(test.sex.map({0: "Female", 1: "Male"}), 2),
    groups=np.concatenate(
        [np.zeros_like(bl_test_probs), np.ones_like(test_probs)]
    ),
    group_names=["Baseline", "Pleiss"],
    title="Mean scores by sex",
    xlabel="Mean score",
    ylabel="Method",
)
eopp_bar

In [50]:
y1_test_probs = test_probs[test.salary==1]
y1_cor_test_probs = test_sds_pred_tranf.scores.flatten()[test.salary==1]

eopp_box = group_box_plots(
    np.concatenate([y1_test_probs, y1_cor_test_probs]),
    np.concatenate([np.zeros_like(y1_test_probs), np.ones_like(y1_cor_test_probs)]),
    np.tile(test.sex[test.salary==1].map(lambda x: "Male and high earner" if x else "Female and high earner"), 2),
    group_names=["Baseline", "Corrected Model"],
)
eopp_box

In [51]:
y1_test_probs = test_probs[test.salary == 1]
y1_cor_test_probs = test_sds_pred_tranf.scores.flatten()[test.salary == 1]

eopp_box = group_box_plots(
    np.concatenate([y1_test_probs, y1_cor_test_probs]),
    np.concatenate(
        [np.zeros_like(y1_test_probs), np.ones_like(y1_cor_test_probs)]
    ),
    np.tile(
        test.sex[test.salary == 1].map(
            lambda x: "Male and high earner" if x else "Female and high earner"
        ),
        2,
    ),
    group_names=["Baseline", "Corrected Model"],
)
eopp_box

In [ ]:
export_plot(eopp_bar, "pleiss-eopp.json")

## Equalised odds

We'll now repeat the process for equalised odds, which requires us changing the underlying cost constraint parameter accordingly, so that the resulting intervention minimises a weighted average between false negative and false positive rate. There are no further parameter choices to be made.

In [52]:
cost_constraint = "weighted"

Learn intervention on validation data.

In [53]:
# Learn parameters to equalize odds and apply to create a new dataset
cpp = CalibratedEqOddsPostprocessing(
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
    cost_constraint=cost_constraint,
    seed=np.random.seed(),
)
cpp = cpp.fit(test_sds, test_sds_pred)

Apply intervention on testing data.

In [54]:
test_sds_pred_tranf = cpp.predict(test_sds_pred)
test_probs = test_sds_pred_tranf.scores.flatten()
test_pred = test_probs > 0.5

## Analyse fairness and accuracy

In [55]:
bl_acc = accuracy(test.salary, bl_test_pred)
acc = accuracy(test.salary, test_pred)

bl_eo = equalized_odds_difference(
    test.salary, bl_test_pred, sensitive_features=test.sex,
)
eo = equalized_odds_difference(
    test.salary, test_pred, sensitive_features=test.sex,
)

bl_calib = calibration(test.salary, bl_test_probs, test.sex)
calib = calibration(test.salary, test_probs, test.sex,)

print(f"Baseline accuracy: {bl_acc:.3f}")
print(f"Accuracy: {acc:.3f}")

print(f"Baseline equalised odds difference: {bl_eo:.3f}")
print(f"Equalised odds difference: {eo:.3f}")

print(f"Baseline calibration: {bl_calib:.3f}")
print(f"Calibration: {calib:.3f}")

Baseline model equalised odds =  0.1048741731426722
Corrected model equalised odds =  0.37386797108620085


In [57]:
eo_bar = group_bar_plots(
    np.concatenate([bl_test_probs, test_probs]),
    np.tile(test.sex.map({0: "Female", 1: "Male"}), 2),
    groups=np.concatenate(
        [np.zeros_like(bl_test_probs), np.ones_like(test_probs)]
    ),
    group_names=["Baseline", "Pleiss"],
    title="Mean scores by sex",
    xlabel="Mean score",
    ylabel="Method",
)
eo_bar

In [58]:
export_plot(eo_bar, "pleiss-eo.json")